In [ ]:
#Innstallerer pakker som er nødvendige

pip install vaderSentiment
pip install nltk
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
#Laster inn datasettet for et av selskapene som er blitt filtrert i R
#Filtrert på artikler som er utgitt utenfor børsens åpningstid og kun omhandler et selskap

AmazonR = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Amazon/Amazon_Ferdig.csv")

In [ ]:
AmazonR.shape

In [ ]:
#Ser hvordan datasettet ser ut

AmazonR.head(3)

In [ ]:
#Laster inn datasettet med selskapets aksjepriser, hentet fra Yahoo Finance direkte i Python

AmazonAksje = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Amazon/AmazonPrice_Label.csv")

In [ ]:
AmazonAksje.shape

In [ ]:
#Ser hvordan datasettet ser ut

AmazonAksje.head(3)

In [ ]:
#Merger datasettene på dato

merge = AmazonR.merge(AmazonAksje, how = "inner", on = "Date")

In [ ]:
#Lager en funksjon som fanger opp subjektivitet
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Lager en funksjon som fanger opp polaritet
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
#Lager to nye kolonner som heter "Subjectivity" og "Polarity"

merge["Subjectivity"] = merge["article_content"].apply(getSubjectivity)
merge["Polarity"] = merge["article_content"].apply(getPolarity)

In [ ]:
#Importerer og laster inn stopword funksjon fra NLTK

import nltk

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
#Lager en ny kolonne i "merge" etter at stopword har blitt innført på artikkel innholdet

merge['article_content_without_stopwords'] = merge['article_content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
#Importer PorterStemmer fra NLTK

from nltk.stem import PorterStemmer

port = PorterStemmer()

In [ ]:
#Lager en funksjon som tar PorterStemming på artikkel innholdet etter stopword
#Lager en ny kolonne som har dette innholdet

def stem_words(article_content_without_stopwords):
    return " ".join([port.stem(word) for word in article_content_without_stopwords.split()])

merge["EtterStemming"] = merge["article_content_without_stopwords"].apply(lambda article_content_without_stopwords: stem_words(article_content_without_stopwords))
merge.head()

In [ ]:
#Lager en funksjon som finner sentiment scoren
def getSIA(EtterStemming):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(EtterStemming)
    return sentiment

In [ ]:
#Laster inn leksikonet fra Vader

nltk.download("vader_lexicon")

In [ ]:
#Henter tak i sentiment scorene for hver dag

compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range(0, len(merge["EtterStemming"])):
    SIA = getSIA(merge["EtterStemming"][i])
    compound.append(SIA["compound"])
    neg.append(SIA["neg"])
    neu.append(SIA["neu"])
    pos.append(SIA["pos"])

In [ ]:
#Lager kolonner i datasettet "merge" slik at alle er samlet

merge["Compound"] = compound
merge["Negative"] = neg
merge["Neutral"] = neu
merge["Positive"] = pos

In [ ]:
#Viser datasettet "merge"

merge.head(3)

In [ ]:
#Beholder nødvendige kolonner

keep_columns = ["Open", "Low", "High", "Date", "Close", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df = merge[keep_columns]
df

In [ ]:
#Gjør datasettet om til en csv fil

df.to_csv("AmazonFørTrening.csv")

In [ ]:
#Filtrerer datasettet i R til train og test sett, der 80% er train og 20% er test.
#Setter sammen alle selskapene innenfor de forskjellige sektorene til en fil.
#Laster inn test settet innenfor sektoren

TeknologiTest = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/6 Big/TestTeknologiVader.csv")

In [ ]:
TeknologiTest.shape

In [ ]:
#Ser hvordan datasetter ser ut

TeknologiTest.head(3)

In [ ]:
#Laster inn treningssettet for sektoren

TeknologiTrain = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/6 Big/TrainTeknologiVader.csv")

In [ ]:
TeknologiTrain.shape

In [ ]:
#Ser hvordan datasettet ser ut

TeknologiTrain.head(3)

In [ ]:
#Lager en liste hvor man bare beholder de kolonnene som er viktige
#For Vader blir det disse:

keep_columns1 = ["Open", "Low", "High", "Volume","Close", "Compound", "Negative", "Neutral", "Positive", "Label"]
df1 = TeknologiTest[keep_columns1]
df1

#For TextBlob blir det disse:

keep_columns1 = ["Open", "Low", "High", "Volume","Close", "Subjectivity", "Polarity", "Label"]
df1 = TeknologiTest[keep_columns1]
df1

In [ ]:
#Lager en liste for datasettet som bare inneholder parameterne som sier noe om sentiment scoren

X = df1
X = np.array(X.drop(["Label", "Open", "Low", "High", "Close", "Volume"], 1))

#Lager liste for om det faktisk er oppgang eller nedgang
y = np.array(df1["Label"])

In [ ]:
#Ser hvordan listen ser ut

X

In [ ]:
#Ser hvordan listen ser ut

y

In [ ]:
#Kaller X for test settet vårt

x_test = X

In [ ]:
#Sjekker lengden på settet

len(x_test)

In [ ]:
#Kaller y for y_test slik at det stemmer med test settet vårt

y_test = y

In [ ]:
#Sjekker lengden på settet

len(y_test)

In [ ]:
#Lager en liste hvor man bare beholder de kolonnene som er viktige
#For Vader blir det disse:

keep_columns2 = ["Open", "Low", "High", "Volume", "Close", "Compound", "Negative", "Neutral", "Positive", "Label"]
df2 = TeknologiTrain[keep_columns2]
df2

#For TextBlob blir det disse:

keep_columns1 = ["Open", "Low", "High", "Volume","Close", "Subjectivity", "Polarity", "Label"]
df2 = TeknologiTest[keep_columns2]
df2

In [ ]:
#Lager en liste for datasettet som bare inneholder parameterne som sier noe om sentiment scoren

X1 = df2
X1 = np.array(X1.drop(["Label", "Open", "Low", "High", "Close", "Volume"], 1))

#Lager liste for om det faktisk er oppgang eller nedgang

y1 = np.array(df2["Label"])

In [ ]:
#Ser hvordan listen ser ut

X1

In [ ]:
#Ser hvordan listen ser ut

y1

In [ ]:
#Kaller X1 for treningssettet vårt

x_train = X1

In [ ]:
#Sjekker lengden på settet

len(x_train)

In [ ]:
#Kaller y1 for y_train slik at det stemmer med treningssettet vårt

y_train = y1

In [ ]:
#Sjekker lengden på settet

len(y_train)

In [ ]:
#Lager en liste som inneholder begge listene vi lagde

frame = [df1, df2]

In [ ]:
#Ser hvordan settet ser ut

result = pd.concat(frame)
result.shape

In [ ]:
#Importerer pakker som trengs for å kjøre deskriptiv statistikk

import statistics
from scipy.stats import skew
from scipy.stats import norm, kurtosis

In [ ]:
#Får frem mesteparten av den deskriptive statistikken som trengs

result.describe(include = "all")

In [ ]:
#Finner medianen på de forskjellige parameterne

statistics.median(result[" "])

In [ ]:
#Finner skjevheten på de forskjellige parameterne

skew(result[" "])

In [ ]:
#Finner kurtosen på de forskjellige parameterne

kurtosis(result[" "])

In [ ]:
#Lager og trener modell for LDA

model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [ ]:
#Viser prediksjonene til modellen

predictions = model.predict(x_test)
predictions

In [ ]:
#Viser om det faktisk er oppgang eller nedgang

y_test

In [ ]:
#Viser modellen sine beregninger

print( classification_report(y_test, predictions))

In [ ]:
#Sjekker sannsynligheten for oppgang og nedgang

Linear = model.predict_proba(x_test)
Linear

In [ ]:
#Importerer pakke som er nødvendig

from sklearn import metrics

In [ ]:
#Lager to kolonner som inneholder den prosentvise sjansen for oppgang og nedgang

column_values_Linear = ["Nedgang-Linear", "Oppgang-Linear"]

df6 = pd.DataFrame(data = Linear,
                  columns = column_values_Linear)

In [ ]:
#Trekker ut den ene kolonnen

extracted_col_Linear = df6["Nedgang-Linear"]
extracted_col_Linear

In [ ]:
#Setter den utrekte kolonnen sammen med datasettet, det samme blir gjort for oppgang

df1 = df1.join(extracted_col_Linear)
df1

In [ ]:
#Lager og trener modell for Naive Bayes Gaussian

from sklearn.naive_bayes import GaussianNB
modelNB = GaussianNB()

In [ ]:
#Trener modellen

modelNB.fit(x_train, y_train)

In [ ]:
#Treffsikkerhet for modellen

modelNB.score(x_test, y_test)

In [ ]:
#Viser prediksjonene til modellen

predictions1 = modelNB.predict(x_test)
predictions1

In [ ]:
#Viser modellen sine beregninger

print( classification_report(y_test, predictions1))

In [ ]:
#Sjekker sannsynligheten for oppgang og nedgang

Bayes = modelNB.predict_proba(x_test)
Bayes

In [ ]:
#Lager kolonner for oppgang og nedgang slik som vist under LDA

In [ ]:
#Lager og trener modell for KNN

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 9)
knn.fit(x_train, y_train)

In [ ]:
#Treffsikkerhet for modellen

knn_score = knn.score(x_test, y_test)
print("Results for KNN Classifier:")
print(knn_score)

In [ ]:
#Viser modellen sine beregninger

from sklearn.metrics import confusion_matrix
cm_knn = confusion_matrix(y_test, y_pred_knn)

In [ ]:
#Viser true negative, false positive, false negative og true positive for modellen

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_knn).ravel()
print(tn, fp, fn, tp)

In [ ]:
#Finner true positive og true negative som prosent

tpr_knn = round(tp/(tp + fn), 4)
tnr_knn = round(tn/(tn + fp), 4)
print(tpr_knn, tnr_knn)

In [ ]:
#Sjekker sannsynligheten for oppgang og nedgang

KNN = knn.predict_proba(x_test)

In [ ]:
#Finner prediksjonen for modellen

predictions_KNN = knn.predict(x_test)
predictions_KNN

In [ ]:
#Viser beregningene til modellen

print( classification_report(y_test, predictions_KNN))

In [ ]:
#Lager egne kolonner for oppgang og nedgang som under LDA og Naive Bayes

In [ ]:
#Innstallerer pakker som er nødvendig

from sklearn import svm

In [ ]:
#Lager og trener modellen for Sector Vector Machine

cls1 = svm.LinearSVC()
cls1.fit(x_train, y_train)

In [ ]:
#Viser prediksjonen til modellen

predSVC1 = cls1.predict(x_test)
predSVC1

In [ ]:
#Viser beregningene til modellen

print( classification_report(y_test, predSVC1))

In [ ]:
#Sjekker sannsynligheten for oppgang og nedgang

Support = cls1._predict_proba_lr(x_test)
Support

In [ ]:
#Lager egne kolonner for oppgang og nedgang slik som under de foregående klassifikatorene

In [ ]:
#Importerer pakker som er nødvendig

from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
#Lager og trener modellen

tree1 = DecisionTreeClassifier(max_depth = 4)
tree1 = tree1.fit(x_train, y_train)

In [ ]:
#Viser informasjon om treet

tree1.get_params()

In [ ]:
#Viser prediksjonen til modellen

predictionsTree = tree1.predict(x_test)
predictionsTree

In [ ]:
#Viser prediksjonen til modellen i prosent

Tree2 = tree1.predict_proba(x_test)
Tree2

In [ ]:
#Viser beregningene til modellen

from sklearn.metrics import classification_report
print(classification_report(y_test, predictionsTree))

In [ ]:
#Fjerner de parameterne som ikke er nødvendige for modellen

feature_names3 = df1.columns.drop(["Open", "Low", "High", "Volume", "Close", "Label", "Nedgang-Bayes", "Oppgang-Bayes", "Nedgang-KNN", "Oppgang-KNN", "Nedgang-Linear", "Oppgang-Linear", "Nedgang-Support", "Oppgang-Support", "Nedgang-Tree", "Oppgang-Tree"], 1)
feature_names3

In [ ]:
#Viser prosentvis viktighet for hver av parameterne som er igjen

tree1.feature_importances_

In [ ]:
#Viser det i tabellform

feature_importance = pd.DataFrame(tree1.feature_importances_, index = feature_names3)
feature_importance

In [ ]:
#Viser det grafisk

feature_importance.head(10).plot(kind = "bar")

In [ ]:
#Lager et decision tree som viser oss valgene og om det er nedgang eller oppgang

from sklearn import tree
from matplotlib import pyplot as plt

fig = plt.figure(figsize = (35,30))
_ = tree.plot_tree(tree1,
                   feature_names = feature_names3,
                   class_names = {0: "Nedgang", 1: "Oppgang"},
                   filled = True,
                   fontsize = 12)

In [ ]:
#Lager egne kolonner for oppgang og nedgang slik som under de foregående klassifikatorene

In [ ]:
#Gjør hele datasettet om til en csv fil før vi gjennomfører en trading strategi ved hjelp av excel